In [4]:
#Authors: Adam Lewandowski, Ivan Sladkov, Patrick English
import numpy as np
import tensorflow as tf
import pandas as pd

In [ ]:
def str_list_to_list(tweets):
    t = tweets.copy()
    import ast 
    t["tweets"] = t["tweets"].apply(lambda x: ast.literal_eval(x))
    return t

tweets = pd.read_csv("data/processed_tweets.csv", encoding = "ISO-8859-1")
tweets = str_list_to_list(tweets)

In [ ]:
print(tweets)